In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Criando um dataset para treino

In [4]:
def classifica_alerta(valor):
  if valor < 0.5:
    return 0 #'Risco de Falta de água'
  elif valor >= 0.5 and valor < 1:
    return 1 #'Alerta'
  elif valor > 1 and valor < 10:
    return 2 #'Vazao normal!'
  else:
    return 3 # Vazao acima da normal

In [ ]:
data_inicio = "2025-01-01 00:00:00"
data_fim = "2025-06-01 00:00:00"
frequencia = "15T" # 15 minutos

vazao_normal_min = 10
vazaon_normal_max = 20

print("Gerando dados sintéticos...")
datas = pd.date_range(start=data_inicio, end=data_fim, freq=frequencia)
df = pd.DataFrame(index=datas)

df['vazao'] = 0.2
perfil_diario = {
    0:0.1, 1:0.1, 2:0.1, 3:0.1, 4:0.2, 5:1.0,
    6:8.0, 7:15.0, 8:10.0, 9:5.0, 10:4.0, 11:7.0,
    12:12.0, 13:10.0, 14:4.0, 15:3.0, 16:3.0, 17:5.0,
    18:10.0, 19:18.0, 20:15.0, 21:8.0, 22:4.0, 23:1.0
}
perfil_semanal = {0:1.0, 1:1.0, 2:1.05, 3:1.0, 4:0.95, 5:1.2, 6:1.15}

horas = df.index.hour
dias_semana = df.index.dayofweek

df['vazao'] += (horas.map(perfil_diario) * dias_semana.map(perfil_semanal))
df['vazao'] += np.random.normal(0, 0.5, len(df)) # Ruído
df['vazao'] = df['vazao'].clip(lower=0) # Sem vazão negativa

df['volume_intervalo'] = df['vazao'] * 15.0


df['volume_acumulado_dia'] = df.groupby(df.index.date)['volume_intervalo'].cumsum()

df['tipo_alerta'] = df['vazao'].apply(classifica_alerta)

df = df.dropna()

df.to_csv('dados_vazao.csv')
print("Dataset gerado: 'dados_vazao_com_volume.csv'")
print(df.head())

KeyError: 'vazao'

## Treinando o modelo de rede neural

Importando as bibliotecas

Extraindo os dados e fazendo a divisão entre treino e teste

In [ ]:
features = ['hora', 'dia_semana', 'vazao', 'volume_acumulado_dia']
target = 'tipo_alerta'

X = df[features].values
y = df[target].values

## Fazendo a normalização dos dados

In [ ]:
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)


Dividindo dados entre treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Treinando o modelo de rede neural

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=[4]), # 4 Entradas
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Ideal para classificação 0,1,2
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5383 - loss: 1.1424
Epoch 2/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8440 - loss: 0.4757
Epoch 3/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8684 - loss: 0.3651
Epoch 4/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8779 - loss: 0.3262
Epoch 5/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8876 - loss: 0.2796
Epoch 6/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9015 - loss: 0.2355
Epoch 7/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9177 - loss: 0.2056
Epoch 8/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9336 - loss: 0.1879
Epoch 9/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9494 - loss: 0.1595
Epoch 10/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9578 - loss: 0.1499
Epoch 11/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9643 - loss: 0.1298
Epoch 12/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

Mensurando valor do Erro Médio Absoluto

In [ ]:
loss, acuracia = model.evaluate(X_test, y_test)
print(f"Acurácia: {acuracia:.4f}")

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9943 - loss: 0.0329
Acurácia: 0.9928


Convertendo o modelo para o ESP32

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('modelo_vazao.tflite', 'wb') as f:
    f.write(tflite_model)

print("Modelo salvo: 'modelo_vazao.tflite'")
print("Não esqueça de rodar: xxd -i modelo_vazao.tflite > modelo_vazao.h")

Saved artifact at '/tmp/tmp10d66wf6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor_16')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  134979297176912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134979297177488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134979297177296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134979297177104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134979297178064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134979297177680: TensorSpec(shape=(), dtype=tf.resource, name=None)
Modelo salvo: 'modelo_vazao.tflite'
Não esqueça de rodar: xxd -i modelo_vazao.tflite > modelo_vazao.h
